# Mestrado em Engenharia Eletrotécnica e Computadores

# Computação Gráfica e Realidade Aumentada-2022/2023

## Basic Shapes and Interactive Scene

#### Alexandre Marques - 2018306089
#### Telmo Cunha - 2018308321

In [2]:
%%html               
<script src="http://is3l.isr.uc.pt/~pm/CGRA/JS/deecshader.js"></script>
<script src="http://is3l.isr.uc.pt/~pm/CGRA/JS/deecapp.js"></script>
<script src="http://is3l.isr.uc.pt/~pm/CGRA/JS/cgraobject.js"></script>
<script src='https://git.io/glm-js.min.js'></script>

In [3]:
#constructor(glcontext)
#this.objetos=[]; // array de objetos que pode conter também objetos compostos
#percorrer todo o array para fazer setshader()
# drawit(vista, projeção, previous)

In [4]:
%%html
<script id="my-vertex-shader" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
uniform mat4 MVP;

varying  vec3 ex_Color;

void main(void) {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

    ex_Color = in_Color;
}
</script>

In [5]:
%%html
<script id="my-fragment-shader" type="x-shader/x-fragment">
precision mediump float;

varying  vec3 ex_Color;

void main(void) {
  
    gl_FragColor = vec4(ex_Color,1.0);
}
</script>

<h2><font color='#ff0084'>Cilindro</font></h2>

In [6]:
%%html
<canvas id="myCanvasCilindro" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">
class cilindro extends CGRAobject{
    constructor(glcontext, r, g, b){
        super(glcontext); // initialize the parent class
        this.numvertices = 120;
        var disco = [];
        var actualvertices = [];
        var ind;
        var colors = [];
        
        for (var i=-1.0; i<=1.0; i+=2){
            if(i == -1.0)
                ind = 0.0;
            else 
                ind = 1.0
            
            var raio = 1.0;
            disco[ind] = [];
            for (var j=-Math.PI/2.0; j<=3*Math.PI/2.0; j+=2.0*Math.PI/10)
                disco[ind] = disco[ind].concat(raio*Math.cos(j),raio*Math.sin(j),i);
            
        }
        // Unir pontos entre discos
        for (var i=0; i<1 ; i++){
            for (var j=0; j<30; j+=3){
                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto do disco acima
                actualvertices = actualvertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]); // 1 ponto
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); // 1 ponto seguinte
                colors = colors.concat(r,g,b);
                
                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto
                actualvertices = actualvertices.concat(disco[i+1][j+3],disco[i+1][j+4],disco[i+1][j+5]); // 1 ponto seguinte
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); // 1 ponto do disco acima
                colors = colors.concat(r,g,b);
            }
            
        }
        // Unir pontos dos discos ao centro
        for (var i=0; i<=1 ; i++){
            for (var j=0; j<30; j+=3){
                if(i==0){
                    actualvertices = actualvertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]);  
                    actualvertices = actualvertices.concat(0,0,-1); 
                    actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]);
                    colors = colors.concat(r,g,b);
                }
                if(i==1){
                    actualvertices = actualvertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]);  
                    actualvertices = actualvertices.concat(0,0,1); 
                    actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]);
                    colors = colors.concat(r,g,b);
                }
                
            }
        }
            
        
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(actualvertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
    }
}

class myappCilindro extends DEECapp{
    counter = 0;
    showt = false;
    
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        // Lets create two triangle objects
        
        this.cilindro = new cilindro(this.gl, [1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0,0.0,1.0]);
        this.cilindro.setShader(this.shaderprog);
        
        this.projectionM = glm.perspective(glm.radians(45),1,0.1,1000);
        this.viewM = glm.lookAt(glm.vec3(0,0,-10),glm.vec3(0,0,1),glm.vec3(0,1,0));
    }
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        var transformation = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(1.0,0.0,0.0)));
        var rotacion = glm.toMat4(glm.angleAxis(glm.radians(45),glm.vec3(1.0,0.0,1.0)));
        this.cilindro.setModelTransformation(transformation['*'](rotacion));
        
        this.cilindro.drawit(this.viewM,this.projectionM);
        

    }
    
}



var app = new myappCilindro('myCanvasCilindro');

app.run();

</script>

<h2><font color='#ff0084'>Cone</font></h2>

In [7]:
%%html
<canvas id="myCanvasCone" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">
class cone extends CGRAobject{
    constructor(glcontext, r, g, b){
        super(glcontext); // initialize the parent class
        this.numvertices = 60;
        var disco = [];
        var actualvertices = [];
        var ind;
        var colors = [];
        for (var i=-1.0; i<=-1.0; i+=1){
            console.log(i);
            ind = 0;
            var raio = 1.0;
            disco[ind] = [];
            for (var j=-Math.PI/2.0; j<=3*Math.PI/2.0; j+=2.0*Math.PI/10)
                disco[ind] = disco[ind].concat(raio*Math.cos(j),raio*Math.sin(j),i);
            
        }
        //# Unir pontos dos discos ao ponto extremo
        for (var j=0; j<30; j+=3){
                actualvertices = actualvertices.concat(disco[0][j],disco[0][j+1],disco[0][j+2]);  
                actualvertices = actualvertices.concat(0,0,1); 
                actualvertices = actualvertices.concat(disco[0][j+3],disco[0][j+4],disco[0][j+5]);
                colors = colors.concat(r,g,b);

        }
        //# Unir pontos dos discos ao centro
        for (var j=0; j<30; j+=3){
                actualvertices = actualvertices.concat(disco[0][j],disco[0][j+1],disco[0][j+2]);  
                actualvertices = actualvertices.concat(0,0,-1); 
                actualvertices = actualvertices.concat(disco[0][j+3],disco[0][j+4],disco[0][j+5]);
                colors = colors.concat(r,g,b);

        }
        
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(actualvertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
    }
}

class myappCone extends DEECapp{
    counter = 0;
    showt = false;
    
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        // Lets create two triangle objects
        
        this.cone = new cone(this.gl, [1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0,0.0,1.0]);
        this.cone.setShader(this.shaderprog);
        
        this.projectionM = glm.perspective(glm.radians(45),1,0.1,1000);
        this.viewM = glm.lookAt(glm.vec3(0,0,-10),glm.vec3(0,0,1),glm.vec3(0,1,0));
    }
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        var transformation = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(1.0,0.0,0.0)));
        var rotacion = glm.toMat4(glm.angleAxis(glm.radians(45),glm.vec3(1.0,0.0,1.0)));
        this.cone.setModelTransformation(transformation['*'](rotacion));
        
        this.cone.drawit(this.viewM,this.projectionM);
        

    }
    
}



var app = new myappCone('myCanvasCone');

app.run();

</script>

<h2><font color='#ff0084'>Cube</font></h2>

In [8]:
%%html
<canvas id="myCanvasCube" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class cube extends CGRAobject{
    constructor(glcontext,r,g,b){
        super(glcontext); // initialize the parent class
        
        this.numvertices = 36;
        var vertices =
            [ [-1.0, -1.0, 1.0],
              [-1.0, 1.0, 1.0],
              [1.0, 1.0, 1.0],
              [1.0, -1.0, 1.0],
              [-1.0, -1.0, -1.0],
              [-1.0, 1.0, -1.0],
              [1.0, 1.0, -1.0],
              [1.0, -1.0, -1.0]]; 
        
        var colors = [];
        
        const indices = [
            0, 1, 3,  1, 3, 2,    //# front
            4, 5, 7,  5, 6, 7,    //# back
            1, 2, 5,  5, 2, 6,   //# top
            0, 3, 4,  3, 4, 7,   //# bottom
            2, 3, 7,  2, 6, 7,   //# right
            0, 1, 4,  1, 4, 5   //# left
          ];
        
        var actualvertices = [];
        for (var j = 0; j < indices.length; ++j){
            actualvertices = actualvertices.concat(vertices[indices[j]]);
            if ((j+1)%3==0)
                colors = colors.concat(r,g,b);
        }
        
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(actualvertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
    }
}

class myappCube extends DEECapp{
    counter = 0;
    
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        // Lets create two triangle objects
        
        this.cube = new cube(this.gl, [1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0,0.0,1.0]);
        this.cube.setShader(this.shaderprog);
        
        this.projectionM = glm.perspective(glm.radians(45),1,0.1,1000);
        this.viewM = glm.lookAt(glm.vec3(0,0,-10),glm.vec3(0,0,1),glm.vec3(0,1,0));
    }
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        var transformation = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(1.0,0.0,0.0)));
        var rotacion = glm.toMat4(glm.angleAxis(glm.radians(45),glm.vec3(1.0,0.0,1.0)));
        this.cube.setModelTransformation(transformation['*'](rotacion));
        
        this.cube.drawit(this.viewM,this.projectionM);

    }
    
}


var app = new myappCube('myCanvasCube');

app.run();

</script>

<h2><font color='#ff0084'>Esfera</font></h2>

In [9]:
%%html
<canvas id="myCanvasEsfera" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class esfera extends CGRAobject{
    constructor(glcontext, r, g, b){
        super(glcontext); // initialize the parent class
        
        this.numvertices = 360;//2160
        var centro = [0.0, 0.0, 0.0];
        var disco = [];
        var colors = [];
        var actualvertices = [];
        const raioConstante = 1.0;
        
        for (var ind=0; ind<=10; ind++){
            disco[ind] = [];
            
            var raio = raioConstante * Math.cos((Math.PI/10*ind) - Math.PI/2);
            
            for (var j=-Math.PI/2.0; j<=3.0*Math.PI; j+=1.0*Math.PI/10.0)
                disco[ind] = disco[ind].concat(raio*Math.cos(j),raio*Math.sin(j),Math.sin((Math.PI/10*ind) - Math.PI/2));
    
        }
        
        for (var i=0; i<disco.length-1 ; i++){
            for (var j=0; j<disco[0].length; j+=3){
                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto do disco acima
                actualvertices = actualvertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]); // 1 ponto
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); // 1 ponto seguinte mesmo disco
                colors = colors.concat(r,g,b);
                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto
                actualvertices = actualvertices.concat(disco[i+1][j+3],disco[i+1][j+4],disco[i+1][j+5]); // 1 ponto seguinte mesmo disco
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); // 1 ponto do disco abaixo
                colors = colors.concat(r,g,b);
            }
        }
        
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(actualvertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
    }
}

class myappEsfera extends DEECapp{
    counter = 0;
    
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        // Lets create two triangle objects
        
        this.esfera = new esfera(this.gl, [1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0,0.0,1.0]);
        this.esfera.setShader(this.shaderprog);
        
        this.projectionM = glm.perspective(glm.radians(45),1,0.1,1000);
        this.viewM = glm.lookAt(glm.vec3(0,0,-10),glm.vec3(0,0,1),glm.vec3(0,1,0));
    }
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        var transformation = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(0.0,0.0,0.0)));
        var rotacion = glm.toMat4(glm.angleAxis(glm.radians(45),glm.vec3(1.0,0.0,0.0)));
        this.esfera.setModelTransformation(transformation['*'](rotacion));
        
        this.esfera.drawit(this.viewM,this.projectionM);

    }
    
}



var app = new myappEsfera('myCanvasEsfera');

app.run();

</script>

<h2><font color='#ff0084'>Objeto de Revolução</font></h2>

In [10]:
%%html
    
<canvas id="myCanvasRevolution" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class revolution extends CGRAobject{
    constructor(glcontext, vertices2d=[], r, g, b){
        super(glcontext);
        this.numvertices = 132;
        var vertices=[];
        var actualvertices=[];
        var i=0;
        var colors=[];
        
        var verticesaux=[];
        var verticeslinha = [];
        var v=[];
        var j=0;
        
        
        // #Ter os vértices em vetores
        for (var i=0; i<vertices2d.length/2; i++){  
            // #vertices2d.length/2 = quantos vetores teremos
            v[i]=[];
            v[i] = v[i].concat(vertices2d[j],vertices2d[j+1],0,1);  // #[x,y,z,1] para rotacionar cada ponto
            j+=2; // #Avançar para o próximo ponto de vertices2d
            //# [x,y,z,1] para glm.vec para poder multiplicar com a matriz
            v[i] = glm.vec4(v[i])
        }
        
        // #Rodar cada vértice (vetor*matriz)
        for (var i=0; i<=10; i+=1){
            var seno = Math.sin(2*Math.PI/10*i);
            var cosseno = Math.cos(2*Math.PI/10*i);
            //var rmat = glm.mat4(cosseno,seno,0,0, -seno,cosseno,0,0, 0,0,1,0, 0,0,0,1);
            var rmat = glm.mat4(cosseno,0,-seno,0,  0,1,0,0,  seno,0,cosseno,0,  0,0,0,1);
            
            // #Inserir vértice rodado em verticesaux
            for (var j=0; j<v.length; j++){
                // #Rotação em verticesaux
                verticesaux[j] = rmat['*'](v[j]);
                for (var k=0;k<3;k++){
                    verticeslinha=verticeslinha.concat(verticesaux[j][k]);
                }
            }
        }
        
        var j;
        for (j=0; j<verticeslinha.length; j+=9){
            actualvertices = actualvertices.concat(verticeslinha[j],verticeslinha[j+1],verticeslinha[j+2]);  // # 1º ponto 1º linha
            actualvertices = actualvertices.concat(verticeslinha[j+3],verticeslinha[j+4],verticeslinha[j+5]); // # 2º ponto 1º linha
            actualvertices = actualvertices.concat(verticeslinha[j+12],verticeslinha[j+13],verticeslinha[j+14]); // # 1º ponto plano 2
            colors = colors.concat(r,g,b);
            
            actualvertices = actualvertices.concat(verticeslinha[j],verticeslinha[j+1],verticeslinha[j+2]);  // # 1 ponto plano 1
            actualvertices = actualvertices.concat(verticeslinha[j+9],verticeslinha[j+10],verticeslinha[j+11]); // # 2º ponto plano 1
            actualvertices = actualvertices.concat(verticeslinha[j+12],verticeslinha[j+13],verticeslinha[j+14]); // # 1º ponto plano 2
            colors = colors.concat(r,g,b);
            
            actualvertices = actualvertices.concat(verticeslinha[j+3],verticeslinha[j+4],verticeslinha[j+5]); // # 2º ponto plano 2
            actualvertices = actualvertices.concat(verticeslinha[j+6],verticeslinha[j+7],verticeslinha[j+8]); // # 1º ponto plano 2
            actualvertices = actualvertices.concat(verticeslinha[j+15],verticeslinha[j+16],verticeslinha[j+17]); // # 2º ponto plano 1
            colors = colors.concat(r,g,b);
            
            actualvertices = actualvertices.concat(verticeslinha[j+3],verticeslinha[j+4],verticeslinha[j+5]); // # 2º ponto plano 2
            actualvertices = actualvertices.concat(verticeslinha[j+12],verticeslinha[j+13],verticeslinha[j+14]); // # 1º ponto plano 2
            actualvertices = actualvertices.concat(verticeslinha[j+15],verticeslinha[j+16],verticeslinha[j+17]); // # 2º ponto plano 1
            colors = colors.concat(r,g,b);
            
        }
        
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(actualvertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        console.log("revoluted object created with " + vertices.length/2 + "vertices") ;
    }
}


class myappRevolution extends DEECapp{
    counter = 0;
    
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        var vertices = [
            0.90, 0.90,
            0.40, 0.00,
            0.00, -0.90];
        
        
        this.rev = new revolution(this.gl, vertices, [1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0,0.0,1.0]);
        this.rev.setShader(this.shaderprog);
        
        this.projectionM = glm.perspective(glm.radians(45),1,0.1,1000);
        this.viewM = glm.lookAt(glm.vec3(0,0,-10),glm.vec3(0,0,1),glm.vec3(0,1,0));
    }
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        var transformation = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(1.0,0.0,0.0)));
        var rotacion = glm.toMat4(glm.angleAxis(glm.radians(45),glm.vec3(1.0,0.0,1.0)));
        this.rev.setModelTransformation(transformation['*'](rotacion));
        
        this.rev.drawit(this.viewM,this.projectionM);
        
        
        //this.circulo.drawit(this.viewM,this.projectionM);
    }
    
}

var app = new myappRevolution('myCanvasRevolution');

app.run();

</script>

<h2><font color='#ff0084'>Objeto de Extrusão</font></h2>

In [11]:
%%html
    
<canvas id="myCanvasExtrusion" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">



class extrusion extends CGRAobject{
    constructor(glcontext, vertices2d=[], r, g, b){
        super(glcontext);
        this.numvertices = 24;
        this.counter=0;
        var vertices=[];
        var colors = [];
        var i=0;
        for(var j=0; j<vertices2d.length+vertices2d.length/2; j++){ // #vertices2d.length+vertices2d.length/2 = quantidade de zz a adicionar
            if((j+1)%3==0) // #cada terceiro elemento será o z, ou seja = -1.0
                vertices[j]=0.0;
            else{
                vertices[j] = vertices2d[i]; // copiar x e y
                i++;
            }
        }

        var vertices2=[];
        for (var j=0; j<vertices.length; j++){
            if ((j+1)%3==0){
                vertices2[j]=1.0;
            }
            else{
                vertices2[j]=vertices[j];
            }
        }

        var actualvertices = [];
        var j=0;
        for (j=0; j<vertices.length-3; j+=3){
            actualvertices = actualvertices.concat(vertices[j],vertices[j+1],vertices[j+2]);  // # 1 ponto plano 1
            actualvertices = actualvertices.concat(vertices[j+3],vertices[j+4],vertices[j+5]); // # 2º ponto plano 1
            actualvertices = actualvertices.concat(vertices2[j],vertices2[j+1],vertices2[j+2]); // # 1º ponto plano 2
            colors = colors.concat(r,g,b);
            
            actualvertices = actualvertices.concat(vertices2[j],vertices2[j+1],vertices2[j+2]); // # 1º ponto plano 2
            actualvertices = actualvertices.concat(vertices2[j+3],vertices2[j+4],vertices2[j+5]); // # 2º ponto plano 2
            actualvertices = actualvertices.concat(vertices[j+3],vertices[j+4],vertices[j+5]); // # 2º ponto plano 1
            colors = colors.concat(r,g,b);
        }
        
        actualvertices = actualvertices.concat(vertices[j],vertices[j+1],vertices[j+2]); // # 2º ponto plano 1
        actualvertices = actualvertices.concat(vertices[0],vertices[0+1],vertices[0+2]);  // # 1 ponto plano 1
        actualvertices = actualvertices.concat(vertices2[j],vertices2[j+1],vertices2[j+2]); // # 2º ponto plano 2
        colors = colors.concat(r,g,b);
        
        actualvertices = actualvertices.concat(vertices[0],vertices[0+1],vertices[0+2]);  // # 1 ponto plano 1
        actualvertices = actualvertices.concat(vertices2[0],vertices2[0+1],vertices2[0+2]); // # 1º ponto plano 2
        actualvertices = actualvertices.concat(vertices2[j],vertices2[j+1],vertices2[j+2]); // # 2º ponto plano 2
        colors = colors.concat(r,g,b);
        
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(actualvertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        console.log("extruded object created with " + vertices.length/2 + "vertices") ;
    }
}

class myappExtrusion extends DEECapp{
    counter = 0;
    
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        var vertices = [
            1.0, 1.0, 
            1.0, -1.0, 
            -1.0, -1.0, 
            -1.0, 1.0];
        
        this.ext = new extrusion(this.gl, vertices, [1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0,0.0,1.0]);
        this.ext.setShader(this.shaderprog);
        
        this.projectionM = glm.perspective(glm.radians(45),1,0.1,1000);
        this.viewM = glm.lookAt(glm.vec3(0,0,-10),glm.vec3(0,0,1),glm.vec3(0,1,0));
    }
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        var rot45 = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(0.0,1.0,0.0)));
        var rotacion = glm.toMat4(glm.angleAxis(glm.radians(-45),glm.vec3(1.0,1.0,0.0)));
        this.ext.setModelTransformation(rot45['*'](rotacion));
        
        this.ext.drawit(this.viewM,this.projectionM);
        
        
    }
    
}

var app = new myappExtrusion('myCanvasExtrusion');

app.run();

</script>


<h2><font color='#ff0084'>Compound Objects</font></h2>

In [14]:
%%html
    
<script id="myapp">

class CompoundObject extends CGRAobject{
    array=[];
    obj;

    ///////////////////////////
    insertObject(objeto, transformation = glm.mat4(1.0)){
        this.obj = objeto;
        this.obj.setModelTransformation(transformation);
        this.array = this.array.concat(this.obj);
    }
    ///////////////////////////
    setShader(shader){
        for (var i=0; i<this.array.length; i++)
            this.array[i].shaderprog = shader;
    }

    ///////////////////////////
    drawit(V,P,parent=glm.mat4(1.0)){
        var trans;
        for(var i=0;i<this.array.length; i++){
            trans = parent['*'](this.array[i].modelMat);
            var aux = this.array[i].modelMat;
            this.array[i].modelMat = trans;
            this.array[i].drawit(V,P);
            this.array[i].modelMat = aux;
        }
    }
    ///////////////////////////
};

</script>

In [15]:
%%html
    
<canvas id="my4Canvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class my4app extends DEECapp{
    counter = 0;
    showt = false;
    
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.test = new CompoundObject;
        
        ////
        // #Criar cada objeto para o compound
        ////
        
        this.cubo = new cube(this.gl, [1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0,0.0,1.0]);
        this.sph = new esfera(this.gl, [1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0,0.0,1.0]);
        
        var scale = glm.mat4(glm.mat3(1));
        var transformation = glm.translate(scale,glm.vec3(-0.5,-0.5,0));
        this.test.insertObject(this.cubo, transformation);
        this.test.insertObject(this.sph);
        this.test.setShader(this.shaderprog);
        
        this.projectionM = glm.perspective(glm.radians(45),1,0.1,1000);
        this.viewM = glm.lookAt(glm.vec3(0,0,-10),glm.vec3(0,0,1),glm.vec3(0,1,0));
    }
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        var rot45 = glm.toMat4(glm.angleAxis(glm.radians(45),glm.vec3(0.0,1.0,1.0)));
        var rotacion = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(1.0,0.0,0.0)));
        //this.test.setModelTransformation(rotacion);
        this.test.drawit(this.viewM,this.projectionM,rotacion);
    }
    
}

var app = new my4app('my4Canvas');

app.run();

</script>

<h2><font color='#ff0084'>Drenas Gerais</font></h2>

In [16]:
%%html
    
<script id="myapp">

class pista extends CGRAobject{
    constructor(glcontext,r,g,b){
        super(glcontext); // initialize the parent class
        
        this.numvertices = 1908;
        var centro = [0.0, 0.0, 0.0];
        var disco = [];
        var actualvertices = [];
        var raio = 10.0;
        var colors=[];
        for (var ind=0; ind<=6; ind++){
            disco[ind] = [];
            for (var j=-Math.PI/2.0; j<=3.0*Math.PI; j+=1.0*Math.PI/15.0)
                disco[ind] = disco[ind].concat(raio*Math.cos(j),0,2*raio*Math.sin(j));
            
            raio+=3;
        }
        var cor = [0.58,0.29,0.0];
        
        for (var i=0; i<disco.length-1 ; i++){
        
            for (var j=0; j<disco[0].length; j+=3){
                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto do disco acima
                actualvertices = actualvertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]); // 1 ponto
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); // 1 ponto seguinte mesmo disco
                colors = colors.concat(cor,cor,cor);
                
                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto
                actualvertices = actualvertices.concat(disco[i+1][j+3],disco[i+1][j+4],disco[i+1][j+5]); // 1 ponto seguinte mesmo disco
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); // 1 ponto do disco abaixo
                colors = colors.concat(cor,cor,cor);
            }
        }
        
        
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(actualvertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
    }
}

</script>

In [17]:
%%html
    
<canvas id="my5Canvas" width="1400" height="700" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class my5app extends DEECapp{
    counter = 0;
    
    initialize(){
        // #no call to super.initialize() this time because
        // #in this case we don't want to use the default shader.
        // #So we need to explicitly prepare it... and we may indeed prepare 
        // #as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        
        // #perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        
        //# ----------------- Barreira -----------------------
        
        this.barreira = new extrusion(this.gl, [0.20, 0.20, 0.55], [0.20, 0.20, 0.55], [0.20, 0.20, 0.55]);
        this.barreira.setShader(this.shaderprog);
        
        //# ------------------- Chão --------------------------
        
        this.chao = new cube(this.gl, [0.0, 0.35, 0.0], [0.0, 0.35, 0.0], [0.0,0.35,0.0]);
        this.chao.setShader(this.shaderprog);
        
        //# ------------------- Céu --------------------------
        
        this.ceu = new cube(this.gl, [0.0, 0.0, 100], [0.0, 0.0, 100], [0.0, 0.0, 100]);
        this.ceu.setShader(this.shaderprog);
        
        //# ------------------- Lua --------------------------
        
        this.lua = new esfera(this.gl, [1.0, 1.0, 0.0], [1.0, 1.0, 0.0], [1.0, 1.0, 0.0]);
        this.lua.setShader(this.shaderprog);
        
        //# ------------------- Track --------------------------
        this.pista = new pista(this.gl);
        this.pista.setShader(this.shaderprog);
        
        //# ------------------ Tree -------------------------------------
        
        this.tree = new CompoundObject;
        
        this.treeCrown = new cone(this.gl,[0.2,0.4,0],[0.2,0.4,0],[0.2,0.4,0]);
        this.treeTrunk = new cilindro(this.gl,[0.2,0.1,0],[0.2,0.1,0],[0.2,0.1,0]);
        
        var grau90 = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        
        var crownPos = glm.vec3(-5.0,10.0,-5.0);
        
        //# Crown
        var scale = glm.scale(glm.vec3(4,4,6));
        var transformation = glm.translate(glm.vec3(crownPos[0],crownPos[1],crownPos[2]));
        this.tree.insertObject(this.treeCrown, transformation['*'](grau90)['*'](scale));
        
        //# Trunk
        var scale = glm.scale(glm.vec3(1,1,5));
        var transformation = glm.translate(glm.vec3(crownPos[0],crownPos[1]-5,crownPos[2]));
        this.tree.insertObject(this.treeTrunk, transformation['*'](grau90)['*'](scale));
        
        this.tree.setShader(this.shaderprog);
    
        //# --------------------------------------------------------------
        //# ------------------ Pine Tree ---------------------------------
        
        this.pineTree = new CompoundObject;
        
        this.pineTreeCrown = new cone(this.gl,[0.2,0.4,0],[0.2,0.4,0],[0.2,0.4,0]);
        this.pineTreeTrunk = new cilindro(this.gl,[0.2,0.1,0],[0.2,0.1,0],[0.2,0.1,0]);
        
        var grau90 = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        
        var crownPos = glm.vec3(-5.0,10.0,-5.0);
        
        //# Crown
        var scale = glm.scale(glm.vec3(4,4,6));
        var transformation = glm.translate(glm.vec3(crownPos[0],crownPos[1],crownPos[2]));
        this.pineTree.insertObject(this.pineTreeCrown, transformation['*'](grau90)['*'](scale));
        
        //# Trunk
        var scale = glm.scale(glm.vec3(1,1,5));
        var transformation = glm.translate(glm.vec3(crownPos[0],crownPos[1]-5,crownPos[2]));
        this.pineTree.insertObject(this.pineTreeTrunk, transformation['*'](grau90)['*'](scale));       
        
        
        this.pineTree.setShader(this.shaderprog);
        //***********************************************************************************************
        //# -------------------------------------------------------------
        
        //# ------------------ Billboard --------------------------------
        this.billboard = new CompoundObject;
        
        this.supportboard1 = new cilindro(this.gl,[0.38,0.38,0.38],[0.38,0.38,0.38],[0.38,0.38,0.38]);
        this.supportboard2 = new cilindro(this.gl,[0.38,0.38,0.38],[0.38,0.38,0.38],[0.38,0.38,0.38]);
        this.board = new cube(this.gl,[0.88,0.88,0.88],[0.88,0.88,0.88],[0.88,0.88,0.88]);
        
        var grau90 = glm.toMat4(glm.angleAxis(Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        
        //# Board
        var scale = glm.scale(glm.vec3(0.2,3,10));
        var transformation = glm.translate(glm.vec3(5.0,10.0,5.0));
        this.billboard.insertObject(this.board, transformation['*'](scale));
        
        //# Support 1
        var scale = glm.scale(glm.vec3(0.2,0.2,5));
        var transformation = glm.translate(glm.vec3(5.0,5.0,-4));
        this.billboard.insertObject(this.supportboard1, transformation['*'](grau90)['*'](scale));
        
        //# Support 1
        var scale = glm.scale(glm.vec3(0.2,0.2,5));
        var transformation = glm.translate(glm.vec3(5.0,5.0,14));
        this.billboard.insertObject(this.supportboard2, transformation['*'](grau90)['*'](scale));
        
        this.billboard.setShader(this.shaderprog);
        
        //---------------------------------------------------------------------------------------
        //#placard
        this.placard = new CompoundObject;
        
        this.supportplacard1 = new cilindro(this.gl,[0.38,0.38,0.38],[0.38,0.38,0.38],[0.38,0.38,0.38]);
        this.boardplacard = new cube(this.gl,[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]);
        
        //# Board
        var scale = glm.scale(glm.vec3(5,3,0.2));
        var transformation = glm.translate(glm.vec3(5.0,10.0,5.0));
        this.placard.insertObject(this.boardplacard, transformation['*'](scale));
        
        //# Support 1
        var scale = glm.scale(glm.vec3(0.5,0.5,2));
        var transformation = glm.translate(glm.vec3(5.0,5.0,5.0));
        this.placard.insertObject(this.supportplacard1, transformation['*'](grau90)['*'](scale));
        
        this.placard.setShader(this.shaderprog);
        
        //**********************************************************************************************************
        //# -------------------------------------------------------------
        
        //# ------------------ Stands ------------------------------------
        this.stand = new CompoundObject;
        
        this.stand1 = new cube(this.gl,[0.5,0.5,0.5],[0.5,0.5,0.5],[0.5,0.5,0.5]);
        this.stand2 = new cube(this.gl,[0.5,0.5,0.5],[0.5,0.5,0.5],[0.5,0.5,0.5]);
        this.stand3 = new cube(this.gl,[0.5,0.5,0.5],[0.5,0.5,0.5],[0.5,0.5,0.5]);
        this.standWall = new cube(this.gl,[0.38,0.38,0.38],[0.38,0.38,0.38],[0.38,0.38,0.38]);
        this.standCover = new cube(this.gl,[0.38,0.38,0.38],[0.38,0.38,0.38],[0.38,0.38,0.38]);
        
        var grau90 = glm.toMat4(glm.angleAxis(Math.PI/2.0,glm.vec3(1.0,0.0,0.0))); 
        var standLength = 30;
        var standPos = glm.vec3(0.0,0.0,0.0);
        
        
        //# Stand 1
        var scale = glm.scale(glm.vec3(standLength,1,5));
        var transformation = glm.translate(glm.vec3(standPos[0],standPos[1],standPos[2]));
        this.stand.insertObject(this.stand1, transformation['*'](scale));
        
        //# Stand 2
        var scale = glm.scale(glm.vec3(standLength,1,3));
        var transformation = glm.translate(glm.vec3(standPos[0],standPos[1]+2,standPos[2]-2));
        this.stand.insertObject(this.stand2, transformation['*'](scale));
        
        //# Stand 3
        var scale = glm.scale(glm.vec3(standLength,1,1));
        var transformation = glm.translate(glm.vec3(standPos[0],standPos[1]+4,standPos[2]-4));
        this.stand.insertObject(this.stand3, transformation['*'](scale));
        
        //# Stand Wall
        var scale = glm.scale(glm.vec3(standLength,6,0.5));
        var transformation = glm.translate(glm.vec3(standPos[0],standPos[1]+5,standPos[2]-5.5));
        this.stand.insertObject(this.standWall, transformation['*'](scale));
        
        //# Stand Cover Roof
        var scale = glm.scale(glm.vec3(standLength,0.5,6));
        var transformation = glm.translate(glm.vec3(standPos[0],standPos[1]+11,standPos[2]));
        this.stand.insertObject(this.standCover, transformation['*'](scale));
        
        this.stand.setShader(this.shaderprog);
        
        this.secondstand = new CompoundObject;
        this.secondstand = this.stand;
        //****************************************************************************************************************
        //# -------------------------------------------------------------
        //# ------------------ Banners --- --------------------------------
        //#Banners 1
        this.finish = new CompoundObject;
        
        this.support3 = new cilindro(this.gl,[0.18,0.18,0.18],[0.18,0.18,0.18],[0.18,0.18,0.18]);
        this.banner = new cube(this.gl,[0.08,0.08,0.08],[0.08,0.08,0.08],[0.08,0.08,0.08]);
        
        var grau90 = glm.toMat4(glm.angleAxis(Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));   
        var boardPos = glm.vec3(0,10,0);
        var bannerScale = 3;
        
        //# Banner
        var scale = glm.scale(glm.vec3(0.2,3,bannerScale));
        var transformation = glm.translate(glm.vec3(0.0,10.0,0.0));
        this.finish.insertObject(this.banner, transformation['*'](scale));
        
        //# Support 1
        var scale = glm.scale(glm.vec3(0.2,0.2,5));
        var transformation = glm.translate(glm.vec3(boardPos[0],boardPos[1]-5,boardPos[2]-bannerScale+1));
        this.finish.insertObject(this.support3, transformation['*'](grau90)['*'](scale));
        
        this.finish.setShader(this.shaderprog);
        
        //-----------------------------------------------------------------------------------
        //#Banners2
        this.finish2 = new CompoundObject;
        
        this.support4 = new cilindro(this.gl,[0.18,0.18,0.18],[0.18,0.18,0.18],[0.18,0.18,0.18]);
        this.banner2 = new cube(this.gl,[0.08,0.08,0.08],[0.08,0.08,0.08],[0.08,0.08,0.08]);
        
        //# Banner
        var scale = glm.scale(glm.vec3(0.2,3,bannerScale));
        var transformation = glm.translate(glm.vec3(0.0,10.0,0.0));
        this.finish2.insertObject(this.banner2, transformation['*'](scale));
        
        //# Support 1
        var scale = glm.scale(glm.vec3(0.2,0.2,5));
        var transformation = glm.translate(glm.vec3(boardPos[0],boardPos[1]-5,boardPos[2]-bannerScale+1));
        this.finish2.insertObject(this.support4, transformation['*'](grau90)['*'](scale));
        
        this.finish2.setShader(this.shaderprog);
        
        //----------------------------------------------------------------------------------------
        //#Banners 3
        this.finish3 = new CompoundObject;
        
        this.support5 = new cilindro(this.gl,[0.18,0.18,0.18],[0.18,0.18,0.18],[0.18,0.18,0.18]);
        this.banner3 = new cube(this.gl,[0.08,0.08,0.08],[0.08,0.08,0.08],[0.08,0.08,0.08]);
        
        //# Banner
        var scale = glm.scale(glm.vec3(0.2,3,bannerScale));
        var transformation = glm.translate(glm.vec3(0.0,10.0,0.0));
        this.finish3.insertObject(this.banner3, transformation['*'](scale));
        
        //# Support 1
        var scale = glm.scale(glm.vec3(0.2,0.2,5));
        var transformation = glm.translate(glm.vec3(boardPos[0],boardPos[1]-5,boardPos[2]-bannerScale+1));
        this.finish3.insertObject(this.support5, transformation['*'](grau90)['*'](scale));
        
        this.finish3.setShader(this.shaderprog);
        
        
        //# -------------------------------------------------------------
        //# ------------------ White Cube Finish ------------------------
        this.whiteCube = new cube(this.gl,[0.88,0.88,0.88],[0.88,0.88,0.88],[0.88,0.88,0.88]);
        this.whiteCube.setShader(this.shaderprog);
        
        //**************************************************************************************************
        //# -------------------------------------------------------------
        //# ------------------ Cars --------------------------------------
        
        this.car2 = new CompoundObject;
        
        this.car = new CompoundObject;
        this.roda1 = new cilindro(this.gl);
        this.roda2 = new cilindro(this.gl);
        this.roda3 = new cilindro(this.gl);
        this.roda4 = new cilindro(this.gl);
        this.jante1 = new cilindro(this.gl,[0.38,0.38,0.38],[0.38,0.38,0.38],[0.38,0.38,0.38]);
        this.jante2 = new cilindro(this.gl,[0.38,0.38,0.38],[0.38,0.38,0.38],[0.38,0.38,0.38]);
        this.jante3 = new cilindro(this.gl,[0.38,0.38,0.38],[0.38,0.38,0.38],[0.38,0.38,0.38]);
        this.jante4 = new cilindro(this.gl,[0.38,0.38,0.38],[0.38,0.38,0.38],[0.38,0.38,0.38]);
        this.base = new cube(this.gl,[0.6,0,0],[0.6,0,0],[0.6,0,0]);
        this.habit = new cube(this.gl,[1,0,0],[1,0,0],[1,0,0]);
        
        this.spoiler= new cube(this.gl,[1,1,1],[1,1,1],[1,1,1]);
        this.supportsp1= new cilindro(this.gl,[1,1,1],[1,1,1],[1,1,1]);
        this.supportsp2 = new cilindro(this.gl,[1,1,1],[1,1,1],[1,1,1]);
        
        var vertices = [
            0.0, 0.0,
            -1.0, 0.00,
            0.00, 1.0];
        
        
        this.habitfront = new revolution(this.gl, vertices, [1,1,1],[1,1,1],[1,1,1]);
        
        var vertices = [
            0.0, 0.0,
            1.0, 0.0,
            0.00, 1.0];
        
        
        this.habitback = new revolution(this.gl, vertices, [1,1,1],[1,1,1],[1,1,1]);
        
        
        this.base2 = new cube(this.gl,[0,0.6,0],[0,0.6,0],[0,0.6,0]);
        this.habit2 = new cube(this.gl,[0,1,0],[0,1,0],[0,1,0]);
        
        var grau90 = glm.toMat4(glm.angleAxis(Math.PI/2.0,glm.vec3(0.0,1.0,0.0)));
        var basePos = glm.vec3(-0.1,1,-0.2);
        
        //# Base Car
        scale = glm.scale(glm.vec3(2.8,1.0,4));
        transformation = glm.translate(glm.vec3(basePos[0],basePos[1],basePos[2]));
        this.car.insertObject(this.base, transformation['*'](scale));
        this.car2.insertObject(this.base2, transformation['*'](scale));
        
        //# Habitaculo
        scale = glm.scale(glm.vec3(2.695,1,2));
        transformation = glm.translate(glm.vec3(basePos[0],basePos[1]+1,basePos[2]));
        this.car.insertObject(this.habit, transformation['*'](scale));
        this.car2.insertObject(this.habit2, transformation['*'](scale));
        
        //# Wheel 1 e Jante 1
        var scale = glm.scale(glm.vec3(1,1,0.5));
        var transformation = glm.translate(glm.vec3(basePos[0]-1.9,basePos[1]-1,basePos[2]-1.8));
        this.car.insertObject(this.roda1, transformation['*'](grau90)['*'](scale));
        this.car2.insertObject(this.roda1, transformation['*'](grau90)['*'](scale));
        var scale = glm.scale(glm.vec3(0.3,0.3,0.05));
        var transformation = glm.translate(glm.vec3(basePos[0]-2.5,basePos[1]-1,basePos[2]-1.8));
        this.car.insertObject(this.jante1, transformation['*'](grau90)['*'](scale));
        this.car2.insertObject(this.jante1, transformation['*'](grau90)['*'](scale));
        
        //# Wheel 2 e Jante 2
        var scale = glm.scale(glm.vec3(1,1,0.5));
        transformation = glm.translate(glm.vec3(basePos[0]+2.1,basePos[1]-1,basePos[2]-1.8));
        this.car.insertObject(this.roda2,transformation['*'](grau90)['*'](scale));
        this.car2.insertObject(this.roda2,transformation['*'](grau90)['*'](scale));
        var scale = glm.scale(glm.vec3(0.3,0.3,0.05));
        var transformation = glm.translate(glm.vec3(basePos[0]+2.7,basePos[1]-1,basePos[2]-1.8));
        this.car.insertObject(this.jante2, transformation['*'](grau90)['*'](scale));
        this.car2.insertObject(this.jante2, transformation['*'](grau90)['*'](scale));
        
        //# Wheel 3 e Jante 3
        var scale = glm.scale(glm.vec3(1,1,0.5));
        transformation = glm.translate(glm.vec3(basePos[0]+2.1,basePos[1]-1,basePos[2]+2.2));
        this.car.insertObject(this.roda3,transformation['*'](grau90)['*'](scale));
        this.car2.insertObject(this.roda3,transformation['*'](grau90)['*'](scale));
        var scale = glm.scale(glm.vec3(0.3,0.3,0.05));
        var transformation = glm.translate(glm.vec3(basePos[0]+2.7,basePos[1]-1,basePos[2]+2.2));
        this.car.insertObject(this.jante3, transformation['*'](grau90)['*'](scale));
        this.car2.insertObject(this.jante3, transformation['*'](grau90)['*'](scale));
        
        //# Wheel 4 e Jante 4
        var scale = glm.scale(glm.vec3(1,1,0.5));
        transformation = glm.translate(glm.vec3(basePos[0]-1.9,basePos[1]-1,basePos[2]+2.2));
        this.car.insertObject(this.roda4,transformation['*'](grau90)['*'](scale));
        this.car2.insertObject(this.roda4,transformation['*'](grau90)['*'](scale));
        var scale = glm.scale(glm.vec3(0.3,0.3,0.05));
        var transformation = glm.translate(glm.vec3(basePos[0]-2.5,basePos[1]-1,basePos[2]+2.2));
        this.car.insertObject(this.jante4, transformation['*'](grau90)['*'](scale));
        this.car2.insertObject(this.jante4, transformation['*'](grau90)['*'](scale));
        
        //#spoiler and support
        var grausupport = glm.toMat4(glm.angleAxis(Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        var scalesup= glm.scale(glm.vec3(0.2,0.2,1.0));
        transformation= glm.translate(glm.vec3(-2.0,2,3.2))
        this.car.insertObject(this.supportsp1,transformation['*'](grausupport)['*'](scalesup));
        this.car2.insertObject(this.supportsp1,transformation['*'](grausupport)['*'](scalesup));
        
        transformation= glm.translate(glm.vec3(2,2,3.2))
        this.car.insertObject(this.supportsp2,transformation['*'](grausupport)['*'](scalesup));
        this.car2.insertObject(this.supportsp2,transformation['*'](grausupport)['*'](scalesup));
        
        var scalesp= glm.scale(glm.vec3(0.5,0.25,4));
        var transformation= glm.translate(glm.vec3(-0.1,3,3.2));
        this.car.insertObject(this.spoiler,transformation['*'](grau90)['*'](scalesp));
        this.car2.insertObject(this.spoiler,transformation['*'](grau90)['*'](scalesp));
        
        //# improvement of the shape of yhe car
        scale = glm.scale(glm.vec3(2.695,1,2));
        transformation = glm.translate(glm.vec3(-0.1,2,-1.9));
        this.car.insertObject(this.habitfront, transformation['*'](scale));
        this.car2.insertObject(this.habitfront, transformation['*'](scale));
        
        scale = glm.scale(glm.vec3(2.695,1,2));
        transformation = glm.translate(glm.vec3(-0.1,2,1.9));
        this.car.insertObject(this.habitback, transformation['*'](scale));
        this.car2.insertObject(this.habitback, transformation['*'](scale));
        
        
        
        this.car.setShader(this.shaderprog);
        
        this.car2.setShader(this.shaderprog);
        
        //*************************************************************************************************************
        //# -------------------------------------------------------------
        
        
        this.projectionM = glm.perspective(glm.radians(90),1,0.1,1000);
        
        // # Posições iniciais nas bancadas, a olhar para o centro da pista
        this.posx = -31.7;
        this.posz = -25.7;
        this.theta = 6*Math.PI/20;
        
        this.viewM = glm.lookAt(glm.vec3(this.posx,0,this.posz),
                                glm.vec3(Math.sin(this.theta)+this.posx,0,
                                         Math.cos(this.theta)+this.posz),
                                glm.vec3(0,1,0));
        
        this.carPos = glm.vec3(0.0,0.0,0.0);
        
        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
        this.viewpoint = true;
        
    }
    
    keyprocess(evt){
        console.log('Key pressed. Keycode:'+evt.keyCode + ' Char: \'' + String.fromCharCode(evt.charCode)+'\'');
         switch (evt.keyCode) {
             case 27:
                alert("You pressed the \"Escape\" key."); 
                break;
             case 112: // #p
                 // # if (this.viewpoint) para não ser possível alterar a câmara do espetador enquanto
                 // # a câmara atual é a POV do carro
             
                 if (this.viewpoint){
                     this.posx += Math.sin(this.theta);
                     this.posz += Math.cos(this.theta);
                 }
                 
                 break;
             case 108: // #l
                 if (this.viewpoint){
                     this.posx -= Math.sin(this.theta);
                     this.posz -= Math.cos(this.theta);
                 }
                 break;
             case 97: // #a 
                 if (this.viewpoint){
                     this.theta+=Math.PI/20;
                 }
                 
                 break;
             case 115: // #s
                 if (this.viewpoint){
                     this.theta-=Math.PI/20;
                 }
                 break;
             case 118: // #v
                 this.viewpoint = !this.viewpoint;
                 break;
        }
    }
    
    render(){
        if (this.viewpoint){ //# POV espectador
        this.viewM = glm.lookAt(glm.vec3(this.posx,5,this.posz),
                                glm.vec3(Math.sin(this.theta)+this.posx,5,
                                         Math.cos(this.theta)+this.posz),
                                glm.vec3(0,1,0));
        }
        else{   //# POV condutor
            this.viewM = glm.lookAt(glm.vec3(this.carPos[0],this.carPos[1]+5,this.carPos[2]),
                                    glm.vec3(this.carPos[0]+Math.cos(-this.counter/20-Math.PI/2),5.5,
                                             this.carPos[2]-Math.sin(-this.counter/20-Math.PI/2)),
                                    glm.vec3(0,1,0));
        }
        
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        var rot180 = glm.toMat4(glm.angleAxis(glm.radians(180),glm.vec3(0.0,1.0,0.0))); //#Rodar 180º para ficar virado para a frente
        //# var rotacion: alteração do ângulo do carro (para alinhar à pista)
        var rotacion = glm.toMat4(glm.angleAxis(-this.counter/20,glm.vec3(0.0,1.0,0.0)));
        //# carPos vai variar de certa forma para percorrer a pista
        this.carPos = glm.vec3(20*Math.cos(this.counter/20),1,2*20*Math.sin(this.counter/20))
        var transformation = glm.translate(this.carPos);

        transformation = (transformation)['*'](rotacion);
        transformation = transformation['*'](rot180);
        // # Se transformation contém a transformação do carro, vamos querer
        // # usar a parte da translação para mover a câmara.
        this.car.drawit(this.viewM,this.projectionM,transformation);
        
        // # Transformações para o 2º carro == 1º carro, mas atrasado(this.counter-10) 
        // # e mais chegado para fora na pista (*22)
        transformation = glm.translate(glm.vec3(22*Math.cos((this.counter-10)/20),1,2*22*Math.sin((this.counter-10)/20)))
                                      ['*'](glm.toMat4(glm.angleAxis(-(this.counter-10)/20,glm.vec3(0.0,1.0,0.0))))
                                      ['*'](glm.toMat4(glm.angleAxis(glm.radians(180),glm.vec3(0.0,1.0,0.0))));
        this.car2.drawit(this.viewM,this.projectionM,transformation);
        
        
        
        this.pista.drawit(this.viewM,this.projectionM);
        
        // # Stands --------------------
        transformation = glm.translate(glm.vec3(-34,0,0))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
        this.stand.drawit(this.viewM,this.projectionM,transformation);
        transformation = glm.translate(glm.vec3(34,0,0))['*'](glm.toMat4(glm.angleAxis(glm.radians(90+180),glm.vec3(0.0,1.0,0.0))));
        this.secondstand.drawit(this.viewM,this.projectionM,transformation);
        
        //*************************************************************************************************
        // # Billboards and Banners ---
        
        //#Billboard  -rigth side
        transformation = glm.translate(glm.vec3(32,0,-50))['*'](glm.toMat4(glm.angleAxis(glm.radians(-10),glm.vec3(0.0,1.0,0.0))));
        this.billboard.drawit(this.viewM,this.projectionM,transformation);
        
        //#placard -right side
        transformation = glm.translate(glm.vec3(0,0,-60))['*'](glm.toMat4(glm.angleAxis(glm.radians(-10),glm.vec3(0.0,1.0,0.0))));
        this.placard.drawit(this.viewM,this.projectionM,transformation);
        
        //# Banners 1
        transformation = glm.translate(glm.vec3(30,0,40))['*'](glm.toMat4(glm.angleAxis(glm.radians(0),glm.vec3(0.0,1.0,0.0))));
        this.finish.drawit(this.viewM,this.projectionM,transformation);
        
        //# Banners 2
        transformation = glm.translate(glm.vec3(25,0,45))['*'](glm.toMat4(glm.angleAxis(glm.radians(0),glm.vec3(0.0,1.0,0.0))));
        this.finish.drawit(this.viewM,this.projectionM,transformation);
        
        //# Banners 3
        transformation = glm.translate(glm.vec3(20,0,50))['*'](glm.toMat4(glm.angleAxis(glm.radians(0),glm.vec3(0.0,1.0,0.0))));
        this.finish.drawit(this.viewM,this.projectionM,transformation);
        
        
        //transformation = glm.translate(glm.vec3(0,12.6,-38))['*'](glm.toMat4(glm.angleAxis(glm.radians(0),glm.vec3(0.0,0.0,1.0))))['*'](glm.scale(glm.vec3(0.3,0.3,18.95)));
        //this.whiteCube.drawit(this.viewM,this.projectionM,transformation);
        
        //transformation = glm.translate(glm.vec3(0,8.3,-38))['*'](glm.toMat4(glm.angleAxis(glm.radians(0),glm.vec3(0.0,0.0,1.0))))['*'](glm.scale(glm.vec3(0.3,0.3,18.95)));
        //this.whiteCube.drawit(this.viewM,this.projectionM,transformation);
        
        //transformation = glm.translate(glm.vec3(0,10.4,-38))['*'](glm.toMat4(glm.angleAxis(glm.radians(0),glm.vec3(0.0,0.0,1.0))))['*'](glm.scale(glm.vec3(0.3,0.3,18.95)));
        //this.whiteCube.drawit(this.viewM,this.projectionM,transformation);
        
            
        //*****************************************************************************************************
        // # Florest ---------------
        //# Trees on the right side of the trak
        var posTree=0;
        for (var i=0; i<8; i++){
            posTree+=20
            transformation = glm.translate(glm.vec3(30-posTree,0,70))['*'](glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0))));
            this.pineTree.drawit(this.viewM, this.projectionM, transformation);
        }        
        // #*****Ground ***** 
        var scale = glm.scale(glm.vec3(500,0.1,500));
        transformation = glm.translate(glm.vec3(0,-0.2,0));
        transformation = transformation['*'](scale);
        this.chao.setModelTransformation(transformation);
        this.chao.drawit(this.viewM,this.projectionM);
        
        // #***** Sun *****
        var scale = glm.scale(glm.vec3(7,7,7));
        transformation = glm.translate(glm.vec3(40,80,80));
        transformation = transformation['*'](scale);
        this.lua.setModelTransformation(transformation);
        this.lua.drawit(this.viewM,this.projectionM);
        
        // #***** Sky *****
        var scale = glm.scale(glm.vec3(500,0.1,500));
        transformation = glm.translate(glm.vec3(0,100,0));
        transformation = transformation['*'](scale);
        this.ceu.setModelTransformation(transformation);
        this.ceu.drawit(this.viewM,this.projectionM);
        
        // #***** BARREIRA *****
        var rotate = glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,0.0,1.0)));
        var scale = glm.scale(glm.vec3(500,500,500));
        transformation = (scale)['*'](rotate);
        this.ceu.setModelTransformation(transformation);
        this.ceu.drawit(this.viewM,this.projectionM);        
    }
    
}

var app = new my5app('my5Canvas');

app.run();

</script>